In [3]:
import warnings
from tqdm import tqdm
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cab
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_recall_fscore_support,roc_curve,auc,roc_auc_score
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import gc
#from featexp import get_univariate_plots#用于特征筛选，需要先安装featexp
warnings.filterwarnings("ignore")
plt.rcParams['font.sans-serif']=['Simhei']
plt.rcParams['axes.unicode_minus']=False
import json

In [4]:
base_info=pd.read_csv('data/train/base_info.csv')#企业的基本信息
annual_report_info=pd.read_csv('data/train/annual_report_info.csv')#企业的年报基本信息
tax_info=pd.read_csv('data/train/tax_info.csv')#企业的纳税信息
change_info=pd.read_csv('data/train/tax_info.csv')#变更信息
news_info=pd.read_csv('data/train/news_info.csv')#舆情信息
other_info=pd.read_csv('data/train/other_info.csv')#其它信息
entprise_info=pd.read_csv('data/train/entprise_info.csv')#企业标注信息{0: 13884, 1: 981}
entprise_evaluate=pd.read_csv('data/entprise_evaluate.csv')#未标注信息

print('base_info shape:',base_info.shape,'id unique:',len(base_info['id'].unique()))
print('annual_report_info shape:',annual_report_info.shape,'id unique:',len(annual_report_info['id'].unique()))
print('tax_info shape:',tax_info.shape,'id unique:',len(tax_info['id'].unique()))
print('change_info shape:',change_info.shape,'id unique:',len(change_info['id'].unique()))
print('news_info shape:',news_info.shape,'id unique:',len(news_info['id'].unique()))
print('other_info shape:',other_info.shape,'id unique:',len(other_info['id'].unique()))
print('entprise_info shape:',entprise_info.shape,'id unique:',len(entprise_info['id'].unique()))
print('entprise_evaluate shape:',entprise_evaluate.shape,'id unique:',len(entprise_evaluate['id'].unique()))

base_info shape: (24865, 33) id unique: 24865
annual_report_info shape: (22550, 23) id unique: 8937
tax_info shape: (29195, 9) id unique: 808
change_info shape: (29195, 9) id unique: 808
news_info shape: (10518, 3) id unique: 927
other_info shape: (1890, 4) id unique: 1888
entprise_info shape: (14865, 2) id unique: 14865
entprise_evaluate shape: (10000, 2) id unique: 10000


In [5]:
base_info['opfrom'].isnull().sum()

0

## 初步数据探索
除了企业的基本信息较为齐全外，其余各表信息均有缺失。很多企业id空缺
训练集总共14865条样本，其中正例:13884,负例981.约为14:1.

In [99]:
#下面看一下具有企业年报信息和纳税信息的企业有多少是非法集资的企业
#首先筛选出非法集资的企业
illegal_id_list=[]
legal_id_list=[]
for index,name_id,flag in entprise_info.itertuples():
    if flag==1:
        illegal_id_list.append(name_id)
    else:
        legal_id_list.append(name_id)
len(legal_id_list),len(illegal_id_list),len(legal_id_list)/len(illegal_id_list)


(13884, 981, 14.15290519877676)

In [100]:
#................ 查看各数据集合中违法和合法的评估信息...............#
def caculate_state__info(legal_id_list,illegal_id_list,name,data):
    state = {'-1':0,'0':0,'1':0}
    for i in data['id'].unique():
        if i in illegal_id_list:
            state['1']+=1
        elif i in legal_id_list:
            state['0']+=1
        else:
            state['-1']+=1
    legal_num,illegal_num = len(legal_id_list),len(illegal_id_list)
    print("具有{}基本信息的企业中，有{}违法；{}合法；{}为测试集".format(name,state['1'],state['0'],state['-1']))
    print("不具有{}基本信息的企业中，有{}违法；{}合法".format(name,illegal_num-state['1'],legal_num-state['0']))
    print("具有{}基本信息的企业中：合法/违法:{}".format(name,(state['0']/state['1'])))
    print("不具有{}基本信息的企业中：合法/违法:{}".format(name,(legal_num-state['0'])/(illegal_num-state['1'])))

# #..................年报基本信息信息数据...................
caculate_state__info(legal_id_list,illegal_id_list,'年报',annual_report_info)
# #...........................纳税信息news_info....................
caculate_state__info(legal_id_list,illegal_id_list,'税收',tax_info) 
#.................. 变更信息 change_info.................
caculate_state__info(legal_id_list,illegal_id_list,'变更信息',change_info) 
#.................. 舆情信息 news_info.................
caculate_state__info(legal_id_list,illegal_id_list,'舆情信息',news_info)
#.................. 其它信息 other_info.................
caculate_state__info(legal_id_list,illegal_id_list,'其它信息',other_info)

具有年报基本信息的企业中，有536违法；2800合法；5601为测试集
不具有年报基本信息的企业中，有445违法；11084合法
具有年报基本信息的企业中：合法/违法:5.223880597014926
不具有年报基本信息的企业中：合法/违法:24.907865168539328
具有税收基本信息的企业中，有75违法；99合法；634为测试集
不具有税收基本信息的企业中，有906违法；13785合法
具有税收基本信息的企业中：合法/违法:1.32
不具有税收基本信息的企业中：合法/违法:15.21523178807947
具有变更信息基本信息的企业中，有75违法；99合法；634为测试集
不具有变更信息基本信息的企业中，有906违法；13785合法
具有变更信息基本信息的企业中：合法/违法:1.32
不具有变更信息基本信息的企业中：合法/违法:15.21523178807947
具有舆情信息基本信息的企业中，有156违法；232合法；539为测试集
不具有舆情信息基本信息的企业中，有825违法；13652合法
具有舆情信息基本信息的企业中：合法/违法:1.4871794871794872
不具有舆情信息基本信息的企业中：合法/违法:16.547878787878787
具有其它信息基本信息的企业中，有139违法；697合法；1052为测试集
不具有其它信息基本信息的企业中，有842违法；13187合法
具有其它信息基本信息的企业中：合法/违法:5.014388489208633
不具有其它信息基本信息的企业中：合法/违法:15.661520190023753


In [101]:
#处理base_info的数据
#空值大于0.5的列都删除掉
annual_report_info_clean=annual_report_info.dropna(thresh=annual_report_info.shape[0]*0.5,how='all',axis=1)
#对object类型进行编码
annual_report_info_clean['BUSSTNAME']=annual_report_info_clean['BUSSTNAME'].fillna("无")
dic = {'无':-1,'开业':0, '歇业':1, '停业':2, '清算':3}
buf = pd.DataFrame()
buf_group = annual_report_info_clean.groupby('BUSSTNAME',sort=False)
for name,group in buf_group:
    group['BUSSTNAME'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
buf=buf.fillna(-1)
#
buf_group = buf.groupby('id',sort=False).agg('mean')
buf=pd.DataFrame(buf_group).reset_index()
annual_report_info_clean=buf.drop(['ANCHEYEAR'],axis=1)
annual_report_info_clean

id  STATE  EMPNUM  \
0     9c7fa510616a683058ce97d0bc768a621cd85ab1e87da2a3    2.0    6.00   
1     9c7fa510616a68309e4badf2a7a3123c0462fb85bf28ef17    2.0   16.00   
2     755db3b5c5f74eb48564a8be9d4a9d7038ed96bc2eea645c    2.0    1.00   
3     da8691b210adb3f6334a7abb56fbae858620b23304f160b5    2.0    1.75   
4     755db3b5c5f74eb46a9abdca3e43a99d07c4aacee3d2cb0d    2.0    1.00   
...                                                ...    ...     ...   
8932  d8071a739aa75a3be9069415a33734b8e3044ccc7b18fe59    2.0    1.00   
8933  d8071a739aa75a3b1e6a0b92c454b72de9a5a524209a60f5    2.0    1.00   
8934  f000950527a6feb6d121d68000403ba01bbfb7813e9f7c51    2.0    1.00   
8935  f000950527a6feb6df93ef17ecca21b6b1122d72587f4360    2.0    0.00   
8936  f000950527a6feb6cbab26794aaba6ab56f85a08923e9677    2.0    3.00   

      EMPNUMSIGN  BUSSTNAME  COLGRANUM  RETSOLNUM  DISPERNUM  UNENUM  \
0           -1.0       -1.0        0.0        0.0        0.0     0.0   
1           -1.0       -1.0        0.0        0.0        0.0     0.0   
2           -1.0       -1.0        0.0        0.0        0.0     0.0   
3           -1.0       -1.0        0.0        0.0        0.0     0.5   
4           -1.0       -1.0        0.0        0.0        0.0     0.0   
...          ...        ...        ...        ...        ...     ...   
8932         2.0        3.0        1.0        0.0        0.0     0.0   
8933         2.0        3.0        0.0        0.0        0.0     0.0   
8934         2.0        3.0       -1.0       -1.0       -1.0    -1.0   
8935         2.0        3.0        0.0        0.0        0.0     0.0   
8936         1.0        3.0        3.0        0.0        0.0     0.0   

      COLEMPLNUM  RETEMPLNUM  DISEMPLNUM  UNEEMPLNUM  WEBSITSIGN  \
0            0.0         0.0         0.0        0.00         2.0   
1            0.0         0.0         0.0        0.00         2.0   
2            0.0         0.0         0.0        0.00         2.0   
3            0.0         0.0         0.0        0.75         2.0   
4            0.0         0.0         0.0        0.00         2.0   
...          ...         ...         ...         ...         ...   
8932         0.0         0.0         0.0        0.00         2.0   
8933         1.0         0.0         0.0        0.00         2.0   
8934        -1.0        -1.0        -1.0       -1.00         2.0   
8935         0.0         0.0         0.0        0.00         1.0   
8936         0.0         0.0         0.0        0.00         2.0   

      FORINVESTSIGN  STOCKTRANSIGN  PUBSTATE  
0              -1.0           -1.0       3.0  
1              -1.0           -1.0       3.0  
2              -1.0           -1.0       3.0  
3              -1.0           -1.0       3.0  
4              -1.0           -1.0       3.0  
...             ...            ...       ...  
8932            2.0            2.0       3.0  
8933            2.0            2.0       3.0  
8934            2.0            2.0       3.0  
8935            2.0            2.0       3.0  
8936            2.0            2.0       1.0  

[8937 rows x 17 columns]

In [102]:
#处理tax数据
tax_info_clean=tax_info.drop(['START_DATE','END_DATE'],axis=1)
tax_info_clean['TAX_CATEGORIES']=tax_info_clean['TAX_CATEGORIES'].fillna("无")
tax_info_clean['TAX_ITEMS']=tax_info_clean['TAX_ITEMS'].fillna("无")
#对object类型进行编码
# tax_info_clean['BUSSTNAME']=tax_infoclean['BUSSTNAME'].fillna("无")
dic={}
cate=tax_info.TAX_CATEGORIES.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf = pd.DataFrame()
buf_group = tax_info_clean.groupby('TAX_CATEGORIES',sort=False)
for name,group in buf_group:
    group['TAX_CATEGORIES'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)

#
dic={}
cate=buf.TAX_ITEMS.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf_group = buf.groupby('TAX_ITEMS',sort=False)
buf = pd.DataFrame()
for name,group in buf_group:
    group['TAX_ITEMS'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
buf=buf.fillna(-1)
#
buf_group = buf.groupby('id',sort=False).agg('mean')
tax_info_clean=pd.DataFrame(buf_group).reset_index()
tax_info_clean

id  TAX_CATEGORIES  \
0    f000950527a6feb6c2f40c9d8477e73a439dfa0897830397        0.566667   
1    f000950527a6feb67cc398bac3bff4a69b4aaa096f975b20        4.254613   
2    f000950527a6feb6891a8c7d5bb8af4bcfaebfe4ccc87efb        3.981744   
3    f000950527a6feb6a4001d4d055bc17b81559375dfc8786d        3.828947   
4    f000950527a6feb6fa6bfc4fe01a9ae5dc3880b78f177c88        1.783831   
..                                                ...             ...   
803  d8071a739aa75a3bf8caa850961981d09933dc800349ea63       13.000000   
804  f000950527a6feb6bd741f7ffa1590df1b85051f3be8cdd1       13.000000   
805  f000950527a6feb613b84f384d8bca7305b7451d3f150040       13.000000   
806  516ab81418ed215d355ce73ceacf29904268b934709af50e       15.000000   
807  f000950527a6feb694d7a6401f1c9e95c1931db540c08e76       16.000000   

      TAX_ITEMS  TAXATION_BASIS  TAX_RATE      DEDUCTION     TAX_AMOUNT  
0     63.966667    4.401121e+04 -0.150153      -0.190667      70.029667  
1     46.780443    8.276959e+07  0.152374    8702.194926  702578.461070  
2     48.626775    1.150764e+06  2.970030   98943.244260    9253.740548  
3     56.022556    7.728593e+06  2.076662  264166.707726   61653.046015  
4     30.991213    4.592809e+08  0.054007    1561.159649  131296.811705  
..          ...             ...       ...            ...            ...  
803  271.000000   -1.000000e+00 -1.000000      -1.000000     500.000000  
804  271.000000   -1.000000e+00 -1.000000      -1.000000      80.000000  
805  271.000000   -1.000000e+00 -1.000000      -1.000000    2000.000000  
806  273.000000   -1.000000e+00 -1.000000      -1.000000    2000.000000  
807  274.000000   -1.000000e+00 -1.000000      -1.000000       4.200000  

[808 rows x 7 columns]

In [103]:
# #处理base_info数据

# 把缺失数量作为一种编码
base_info_clean = base_info
base_info_clean['nan_num'] = base_info.isnull().sum(axis=1)
base_info_clean = base_info_clean.drop(['opscope','opfrom','opto','dom'],axis=1)


#............................对object类型进行编码...............................
base_info_clean['industryphy']=base_info_clean['industryphy'].fillna("无")
# base_info_clean['dom']=base_info_clean['dom'].fillna("无")
base_info_clean['opform']=base_info_clean['opform'].fillna("无")
base_info_clean['oploc']=base_info_clean['oploc'].fillna("无")

dic={}
cate=base_info_clean.industryphy.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf = pd.DataFrame()
buf_group = base_info_clean.groupby('industryphy',sort=False)
for name,group in buf_group:
    group['industryphy'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
print('finished 1....')
#
# dic={}
# cate=buf.dom.unique()
# for i in range(len(cate)):
#     dic[cate[i]]=i

# buf_group = buf.groupby('dom',sort=False)
# buf = pd.DataFrame()
# for name,group in buf_group:
#     group['dom'] = dic[name]
#     buf = pd.concat([buf,group],ignore_index=True)
# print('finished 2....')
#
dic={}
cate=buf.opform.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf_group = buf.groupby('opform',sort=False)
buf = pd.DataFrame()
for name,group in buf_group:
    group['opform'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
print('finished 3....')
#
dic={}
cate=buf.oploc.unique()
for i in range(len(cate)):
    dic[cate[i]]=i

buf_group = buf.groupby('oploc',sort=False)
buf = pd.DataFrame()
for name,group in buf_group:
    group['oploc'] = dic[name]
    buf = pd.concat([buf,group],ignore_index=True)
print('finished 4....')

buf=buf.fillna(-1)

buf_group = buf.groupby('id',sort=False).agg('mean')
base_info_clean=pd.DataFrame(buf_group).reset_index()

print('编码完毕.................')

finished 1....
finished 3....
finished 4....
编码完毕.................


In [104]:
#........................分桶.................................
def bucket(name,bucket_len):
    gap_list=[base_info_clean[name].quantile(i/bucket_len) for i in range(bucket_len+1)]
    len_data=len(base_info_clean[name])
    new_col=[]
    for i in base_info_clean[name].values:
        for j in range(len(gap_list)):
            if gap_list[j]>=i:
                encode=j
                break
        new_col.append(encode)
    return new_col
#注册资本_实缴资本
base_info_clean['regcap_reccap']=base_info_clean['regcap']-base_info_clean['reccap']
#注册资本分桶
base_info_clean['regcap']=base_info_clean['regcap'].fillna(base_info_clean['regcap'].median())
base_info_clean['bucket_regcap']=bucket('regcap',5)
#实缴资本分桶
base_info_clean['reccap']=base_info_clean['reccap'].fillna(base_info_clean['reccap'].median())
base_info_clean['bucket_reccap']=bucket('reccap',5)
#注册资本_实缴资本分桶
base_info_clean['regcap_reccap']=base_info_clean['regcap_reccap'].fillna(base_info_clean['regcap_reccap'].median())
base_info_clean['bucket_regcap_reccap']=bucket('regcap_reccap',5)

# 缺失数量分桶
base_info_clean['bucket_nan_num'] = bucket('regcap_reccap',5)
print('分桶完毕.................')
#.............................交叉.........................
#作两个特征的交叉
def cross_two(name_1,name_2):
    new_col=[]
    encode=0
    dic={}
    val_1=base_info[name_1]
    val_2=base_info[name_2]
    for i in tqdm(range(len(val_1))):
        tmp=str(val_1[i])+'_'+str(val_2[i])
        if tmp in dic:
            new_col.append(dic[tmp])
        else:
            dic[tmp]=encode
            new_col.append(encode)
            encode+=1
    return new_col
#作企业类型-小类的交叉特征
base_info_clean['enttypegb']=base_info_clean['enttypegb'].fillna("无")
base_info_clean['enttypeitem']=base_info_clean['enttypeitem'].fillna("无")
new_col=cross_two('enttypegb','enttypeitem')#作企业类型-小类的交叉特征
base_info_clean['enttypegb_enttypeitem']=new_col
#
#行业类别-细类的交叉特征
base_info_clean['industryphy']=base_info_clean['industryphy'].fillna("无")
base_info_clean['industryco']=base_info_clean['industryco'].fillna("无")
new_col=cross_two('industryphy','industryco')#作企业类型-小类的交叉特征
base_info_clean['industryphy_industryco']=new_col
print('交叉特征完毕.................')

100%|██████████| 24865/24865 [00:00<00:00, 113427.50it/s]交叉特征完毕.................



In [105]:
cat_features=['industryphy','opform','oploc','bucket_regcap',
              'bucket_reccap','bucket_regcap_reccap',
              'enttypegb','enttypeitem','enttypegb_enttypeitem',
              'industryphy','industryco','industryphy_industryco',
              'adbusign','townsign','regtype','TAX_CATEGORIES'
             ]


In [106]:
#暂时可以利用企业基本信息，企业纳税信息，企业年度财报信息做义工merge然后进行我们的分类工作
all_data=base_info_clean.merge(annual_report_info_clean,how='outer')
all_data=all_data.merge(tax_info_clean,how='outer')
all_data=all_data.fillna(-1)
all_data.shape,base_info.shape,annual_report_info.shape,tax_info.shape

((24865, 59), (24865, 34), (22550, 23), (29195, 9))

In [107]:
all_data[cat_features]=all_data[cat_features].astype(int)

In [108]:
#train_data=all_data[all_data['id'].isin(entprise_info['id'].unique().tolist())]
#train_data=train_data.reset_index(drop=True)
train_df=all_data.merge(entprise_info)
train_data=train_df.drop(['id','label'],axis=1)
kind=train_df['label']
test_df=all_data[all_data['id'].isin(entprise_evaluate['id'].unique().tolist())]
test_df=test_df.reset_index(drop=True)
test_data=test_df.drop(['id'],axis=1)
train_data.shape,test_data.shape

((14865, 58), (10000, 58))

In [109]:
#特征筛选
#frt_select=[
#  'industryco','industryphy','regcap','reccap',
#  'regcap_reccap','enttypegb','enttypeitem','adbusign','TAX_CATEGORIES',
#  'townsign','empnum','TAX_AMOUNT','industryphy_industryco',
#  'venind','enttypeminu','EMPNUM','COLGRANUM',
#  'dom','jobid','enttypegb_enttypeitem','parnum','bucket_regcap',
#  'exenum','opform','bucket_reccap','oplocdistrict','TAX_ITEMS',
#  'bucket_regcap_reccap','orgid','COLEMPLNUM','FORINVESTSIGN',
#  'BUSSTNAME','compform','regtype','RETEMPLNUM','STATE','EMPNUMSIGN','enttype','UNEEMPLNUM',
# ]
# train_data=train_data[frt_select]
# test_data=test_data[frt_select]
# cat_features=list(set(frt_select).intersection(set(cat_features)))
# cat_features

In [110]:
def eval_score(y_test,y_pre):
    _,_,f_class,_=precision_recall_fscore_support(y_true=y_test,y_pred=y_pre,labels=[0,1],average=None)
    fper_class={'合法':f_class[0],'违法':f_class[1],'f1':f1_score(y_test,y_pre)}
    return fper_class
#
def k_fold_serachParmaters(model,train_val_data,train_val_kind):
    mean_f1=0
    mean_f1Train=0
    n_splits=5
    sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
    for train, test in sk.split(train_val_data, train_val_kind):
        x_train = train_val_data.iloc[train]
        y_train = train_val_kind.iloc[train]
        x_test = train_val_data.iloc[test]
        y_test = train_val_kind.iloc[test]

        model.fit(x_train, y_train)
        pred = model.predict(x_test)
        fper_class =  eval_score(y_test,pred)
        mean_f1+=fper_class['f1']/n_splits
        #print(fper_class)
        
        pred_Train = model.predict(x_train)
        fper_class_train =  eval_score(y_train,pred_Train)
        mean_f1Train+=fper_class_train['f1']/n_splits
    #print('mean valf1:',mean_f1)
    #print('mean trainf1:',mean_f1Train)
    return mean_f1

In [111]:
def search_param(iter_cnt,lr,max_depth):
    clf=cab.CatBoostClassifier(iterations=iter_cnt
                              ,learning_rate=lr
                              ,depth=max_depth
                              ,silent=True
                              ,thread_count=8
                              ,task_type='CPU'
                              ,cat_features=cat_features
                              )
    mean_f1=k_fold_serachParmaters(clf,train_data,kind)
    return mean_f1

#搜索最佳参数
param=[]
best=0
for iter_cnt in [55,60,70]:
    print('iter_cnt:',iter_cnt)
    for lr in [0.050,0.055,0.060,0.065]:
        for max_depth in [5,6,7,8]:
            mean_f1=search_param(iter_cnt,lr,max_depth)
            if mean_f1>best:
                param=[iter_cnt,lr,max_depth]
                best=mean_f1
                print(param,best)



iter_cnt: 55
[55, 0.05, 5] 0.823627697409949
[55, 0.05, 6] 0.8296895830730766
[55, 0.055, 7] 0.8323499588860255
[55, 0.06, 8] 0.8382743912370926
[55, 0.065, 8] 0.8390257168838791
iter_cnt: 60
iter_cnt: 70


In [131]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import learning_curve, GridSearchCV

def search_param(n_estimators,max_depth,min_samples_split):
    rf = RandomForestClassifier(oob_score=True, random_state=2020,
                    n_estimators= n_estimators,max_depth=max_depth,min_samples_split=min_samples_split)
    mean_f1=k_fold_serachParmaters(rf,train_data,kind)
    return mean_f1

#搜索最佳参数
param=[]
best=0
for n_estimators in [57,58,59,60,65]:
    print('n_estimators:',n_estimators)
    for min_samples_split in [6,7,8,10,13,15]:
        for max_depth in [11,12,13,15]:
            mean_f1=search_param(n_estimators,max_depth,min_samples_split)
            if mean_f1>best:
                param=[n_estimators,min_samples_split,max_depth]
                best=mean_f1
                print(param,best)

n_estimators: 57
[57, 6, 11] 0.822567026484206


KeyboardInterrupt: 

In [123]:
def predict(model, train_data, train_label, test_data,n_splits = 5, shuffle = True,random_state=2020):
    mean_f1 = 0
    answers = []
    feature_importance_list = []
    sk = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=2020)
    for i, (train, valid) in enumerate(sk.split(train_data, train_label)):
        x_train = train_data.iloc[train]
        y_train = train_label.iloc[train]
        x_valid = train_data.iloc[valid]
        y_valid = train_label.iloc[valid]

        model.fit(x_train, y_train)
        pred_cab = model.predict(x_valid)
        f1_score =  eval_score(y_valid,pred_cab)['f1']
        print('model = {} 第{}次验证的f1:{}'.format(model,i+1,f1_score))
        feature_importance = pd.DataFrame({
        'column': train_data.columns,
        'importance': model.feature_importances_,
        })
        feature_importance_list.append(feature_importance)
        mean_f1 += f1_score/n_splits
        ans = model.predict_proba(test_data)
        answers.append(ans)
    print('mean f1:',mean_f1)
    feature_importances = pd.concat(feature_importance_list)
    feature_importances = feature_importances.groupby(
    'column')['importance'].agg('mean').sort_values(ascending=False).reset_index()
    return np.sqrt(sum(np.array(answers)**2)/n_splits)[:,1], feature_importances


In [124]:
clf=cab.CatBoostClassifier(iterations=55
                              ,learning_rate=0.065
                              ,depth=8
                              ,silent=True
                              ,thread_count=8
                              ,task_type='CPU'
                              ,cat_features=cat_features
                              )

rf = RandomForestClassifier(oob_score=True, random_state=2020,
            n_estimators= 60,max_depth=13,min_samples_split=6)


rf_score,rf_feature_importances = predict(rf,train_data,kind,test_data)
cat_score,cat_feature_importances = predict(clf,train_data,kind,test_data)

model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第1次验证的f1:0.8465346534653465
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第2次验证的f1:0.8050632911392406
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第3次验证的f1:0.846153846153846
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第4次验证的f1:0.8295165394402036
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第5次验证的f1:0.8142493638676845
mean f1: 0.8283035388132642
model = <catboost.core.CatBoostClassifier object at 0x1a28511550> 第1次验证的f1:0.8557213930348259
model = <catboost.core.CatBoostCla

In [125]:
#
final_score = (rf_score + cat_score)/2.0
test_df['score'] = final_score #可选:fina_persudo是伪标签的预测结果
submit_csv=test_df[['id','score']]
submit_csv.to_csv('submit.csv',index=False)

In [126]:

rf_feature_importances.head()




column  importance
0   industryco    0.223530
1       exenum    0.082331
2  industryphy    0.079328
3  enttypeitem    0.064778
4  enttypeminu    0.057838

In [127]:
topk_feature_names = []
topk_feature_names.extend(list(cat_feature_importances['column'].values[:20]))
topk_feature_names.extend(list(rf_feature_importances['column'].values[:20]))
topk_feature_names = set(topk_feature_names)
print(topk_feature_names)

{'bucket_regcap', 'enttypegb', 'bucket_regcap_reccap', 'empnum', 'townsign', 'FORINVESTSIGN', 'enttypeitem', 'industryphy_industryco', 'industryphy', 'bucket_reccap', 'industryco', 'nan_num', 'orgid', 'bucket_nan_num', 'regcap', 'enttypeminu', 'enttype', 'EMPNUM', 'jobid', 'reccap', 'parnum', 'oplocdistrict', 'regcap_reccap', 'exenum'}


In [118]:
topk_feature_names

{'EMPNUM',
 'FORINVESTSIGN',
 'bucket_nan_num',
 'bucket_reccap',
 'bucket_regcap',
 'bucket_regcap_reccap',
 'empnum',
 'enttype',
 'enttypegb',
 'enttypegb_enttypeitem',
 'enttypeitem',
 'enttypeminu',
 'exenum',
 'industryco',
 'industryphy',
 'industryphy_industryco',
 'jobid',
 'nan_num',
 'oplocdistrict',
 'orgid',
 'parnum',
 'reccap',
 'regcap',
 'regcap_reccap',
 'townsign'}

In [128]:
cat_features=['industryphy','opform','oploc','bucket_regcap',
              'bucket_reccap','bucket_regcap_reccap',
              'enttypegb','enttypeitem','enttypegb_enttypeitem',
              'industryphy','industryco','industryphy_industryco',
              'adbusign','townsign','regtype','TAX_CATEGORIES'
             ]

cat_features = topk_feature_names.intersection(cat_features)

In [129]:
cat_features

{'bucket_reccap',
 'bucket_regcap',
 'bucket_regcap_reccap',
 'enttypegb',
 'enttypeitem',
 'industryco',
 'industryphy',
 'industryphy_industryco',
 'townsign'}

In [130]:
rf_score,rf_feature_importances = predict(rf,train_data[topk_feature_names],kind,test_data[topk_feature_names])
clf=cab.CatBoostClassifier(iterations=69
                              ,learning_rate=0.050
                              ,depth=8
                              ,silent=True
                              ,thread_count=8
                              ,task_type='CPU'
                              ,cat_features=cat_features
                              )
cat_score,cat_feature_importances = predict(clf,train_data[topk_feature_names],kind,test_data[topk_feature_names])

model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第1次验证的f1:0.8486352357320099
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第2次验证的f1:0.7979274611398963
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第3次验证的f1:0.8407960199004973
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第4次验证的f1:0.8354430379746836
model = RandomForestClassifier(max_depth=13, min_samples_split=6, n_estimators=60,
                       oob_score=True, random_state=2020) 第5次验证的f1:0.8168316831683168
mean f1: 0.8279266875830809
model = <catboost.core.CatBoostClassifier object at 0x1a27d18e50> 第1次验证的f1:0.8535980148883374
model = <catboost.core.CatBoostCl

In [122]:
#
merge_score = (rf_score + cat_score)/2.0
test_df['score'] = merge_score #可选:fina_persudo是伪标签的预测结果
submit_csv=test_df[['id','score']]
submit_csv.to_csv('merge_submit.csv',index=False)